In [ ]:
# =============================
# EXTRACTION XML-TEI DE HAL : creation de fichiers xml qui seront utilisés pour la suite du script
# =============================
import os
import time
import requests
import xml.etree.ElementTree as ET
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# =============================
# PARAMÈTRES GÉNÉRAUX
# =============================
NOM_COLLECTION = "INRIA"
ANNEES = list(range(2018, 2025))  # années à traiter
ROWS = 1000  # nombre de notices par requête
OUT_DIR = "hal_xml_tei"
MAX_WORKERS = 4  # nombre de threads
os.makedirs(OUT_DIR, exist_ok=True)

BASE_URL = f"https://api.archives-ouvertes.fr/search/{NOM_COLLECTION}"

# =============================
# FONCTION PRINCIPALE PAR ANNÉE
# =============================
def harvest_year(year: int):
    try:
        print(f"\n🌱 Démarrage extraction HAL pour {year}")

        # --- récupérer le nombre total de notices ---
        count_params = {
            "q": f"publicationDateY_i:{year}",
            "wt": "json",
            "rows": 0
        }
        r_count = requests.get(BASE_URL, params=count_params, timeout=60)
        r_count.raise_for_status()
        total = r_count.json()["response"]["numFound"]

        if total == 0:
            print(f"⚠️  {year} : aucune notice")
            return

        print(f"📚 {year} : {total} notices HAL")

        cursor = "*"
        page_num = 1

        with tqdm(total=total, desc=f"{year}", position=year % 10) as pbar:
            while True:
                params = {
                    "q": f"publicationDateY_i:{year}",
                    "wt": "xml-tei",
                    "rows": ROWS,
                    "sort": "docid asc",
                    "cursorMark": cursor
                }

                r = requests.get(BASE_URL, params=params, timeout=120)
                r.raise_for_status()
                text = r.text

                # Nom du fichier par page
                out_file_page = os.path.join(OUT_DIR, f"HAL_{year}_page{page_num}.xml")
                with open(out_file_page, "w", encoding="utf-8") as f_out:
                    f_out.write(text)

                # Lire next cursorMark dans le XML TEI
                try:
                    tree = ET.fromstring(text)
                    next_cursor_mark = tree.attrib.get("next")
                except ET.ParseError:
                    print(f"❌ ERREUR parsing XML pour {year}, page {page_num}")
                    break

                pbar.update(min(ROWS, total - pbar.n))
                time.sleep(0.2)  # politesse HAL

                if not next_cursor_mark or next_cursor_mark == cursor:
                    break

                cursor = next_cursor_mark
                page_num += 1

        print(f"✅ {year} terminé, {page_num} pages extraites")

    except Exception as e:
        print(f"❌ ERREUR pour {year} : {e}")

# =============================
# LANCEMENT PARALLÈLE
# =============================
if __name__ == "__main__":
    print("🚀 Lancement extraction HAL (threads, par année)\n")

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(harvest_year, y): y for y in ANNEES}

        for future in as_completed(futures):
            year = futures[future]
            try:
                future.result()
            except Exception as e:
                print(f"❌ Erreur pour {year} :", e)

    print("\n🎉 Extraction HAL terminée")

# Comptage du nombre de notices récupérées par années, pour contrôle


import glob
import os

def count_biblfull_in_file(xml_file):
    with open(xml_file, "r", encoding="utf-8") as f:
        text = f.read()
    return text.count("<biblFull>")

grand_total = 0

for year in range(2018, 2025):
    # tous les fichiers pour cette année
    files = sorted(glob.glob(os.path.join("hal_xml_tei", f"HAL_{year}_page*.xml")))
    total = 0
    for xml_file in files:
        n = count_biblfull_in_file(xml_file)
        total += n
    grand_total += total
    print(f"{year} : {total} notices (texte brut, toutes pages)")

print(f"\n📚 Total général (2018-2024) : {grand_total} notices")


# Temps de traitement : 1mn 64sec.


In [ ]:
#=============================================================================
# exploitation des données des fichiers xml
# Creation d'un df structure et d'un df auteurs à partir de toutes les publications
#=============================================================================
import os
import re
import pandas as pd
import xml.etree.ElementTree as ET

# ==============================
# PARAMÈTRES
# ==============================
XML_DIR = "hal_xml_tei"  # dossier contenant tous les fichiers XML
tei_ns = "http://www.tei-c.org/ns/1.0"
xml_ns = "http://www.w3.org/XML/1998/namespace"

# ==============================
# FONCTIONS
# ==============================

def extract_structures(text):
    """Extrait les structures d’un XML TEI (string)"""
    listorg_match = re.search(
        r'<listOrg[^>]*type="structures">(.*?)</listOrg>',
        text,
        re.DOTALL
    )
    if not listorg_match:
        return pd.DataFrame()

    listorg_text = listorg_match.group(1)
    orgs_text = re.findall(r'(<org[^>]*>.*?</org>)', listorg_text, re.DOTALL)

    structures = []
    for org_block in orgs_text:
        xml_id = re.search(r'xml:id="struct-(\d+)"', org_block)
        valid_s = re.search(r'<org[^>]*status="([^"]+)"', org_block).group(1)
        type_org = re.search(r'<org[^>]*type="([^"]+)"', org_block)
        lenom = re.search(r'<orgName>([^<]+)</orgName>', org_block)
        lacronyme = re.search(r'<orgName type="acronym">([^<]+)</orgName>', org_block)
        ladresse = re.findall(r'<addrLine>([^<]+)</addrLine>', org_block)
        lepays = re.search(r'<country key="([^"]+)"', org_block)
        lesrelations = re.findall(r'<relation[^>]*active="#struct-(\d+)"', org_block)
        rnsr = re.search(r'<idno[^>]*type="RNSR">([^<]+)</idno>', org_block)
        ror_id = re.search(r'<idno[^>]*type="ROR">([^<]+)</idno>', org_block)
        structures.append({
            "id_Aurehal": xml_id.group(1) if xml_id else "",
            "type": type_org.group(1) if type_org else "",
            "statut": valid_s if valid_s else "",
            "name": lenom.group(1) if lenom else "",
            "acronym": lacronyme.group(1) if lacronyme else "",
            "address": " ".join(ladresse) if ladresse else "",
            "country": lepays.group(1) if lepays else "",
            "RNSR": rnsr.group(1) if rnsr else "",
            "ror_id": ror_id.group(1) if ror_id else "",
            "relations": ",".join(lesrelations)
        })
    return pd.DataFrame(structures)


def extract_publis(tree, xml_file):
    """Extrait les publications et auteurs d’un XML TEI et ajoute le nom du fichier source"""
    def T(tag): return f"{{{tei_ns}}}{tag}"
    def get_full_text(elem):
        return "".join(elem.itertext()).strip() if elem is not None else ""

    records = []
    root = tree.getroot()
    biblfulls = root.findall(f".//{T('body')}//{T('biblFull')}")

    for biblfull in biblfulls:
        halID_elem = biblfull.find(f".//{T('publicationStmt')}/{T('idno')}[@type='halId']")
        halID = halID_elem.text if halID_elem is not None else ""

        date_elem = biblfull.find(f".//{T('imprint')}/{T('date')}[@type='datePub']")
        date_prod = biblfull.find(f".//{T('edition')}/{T('date')}[@type='whenProduced']")
        year = (date_elem.text if date_elem is not None else
                date_prod.text if date_prod is not None else "")[:4]

        keywords = biblfull.findall(f".//{T('keywords')}/{T('term')}")
        keywords_str = ";".join(" ".join(k.text.split()) for k in keywords if k.text)

        domains = biblfull.findall(f".//{T('classCode')}[@scheme='halDomain']")
        hal_domain = ";".join(d.text.strip() for d in domains if d.text)

        # Déclare le namespace TEI
        namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}

        # Type de doc
        hal_typology_elem = biblfull.find(f".//{T('classCode')}[@scheme='halTypology']")
        hal_typology_value = hal_typology_elem.get('n') if hal_typology_elem is not None else ""

        abstract_elem = biblfull.find(f".//{T('abstract')}[@{{{xml_ns}}}lang='en']")
        if abstract_elem is None:
            abstract_elem = biblfull.find(f".//{T('abstract')}[@{{{xml_ns}}}lang='fr']")
        abstract_str = get_full_text(abstract_elem)

        


        authors = biblfull.findall(f".//{T('author')}")
        for author in authors or [None]:
            if author is None:
                author_name = "Unknown"
                aff_ids = [""]
            else:
                fn = author.find(f".//{T('forename')}")
                sn = author.find(f".//{T('surname')}")
                author_name = f"{sn.text if sn is not None else 'Unknown'}, {fn.text if fn is not None else 'Unknown'}"
                affiliations = author.findall(f".//{T('affiliation')}")
                aff_ids = [aff.get("ref").lstrip("#struct-") for aff in affiliations if aff.get("ref")] or [""]

            for aff_id in aff_ids:
                records.append({
                    "halID": halID,
                    "year": year,
                    "keywords": keywords_str,
                    "hal_domain": hal_domain,
                    "abstract": abstract_str,
                    "author": author_name,
                    "id_Aurehal": aff_id,
                    "DocType": hal_typology_value,
                    "xml_file": xml_file  # colonne source
                })

    return pd.DataFrame(records)


# ==============================
# BOUCLE SUR TOUS LES FICHIERS DU DOSSIER
# ==============================

all_structures = []
all_publis = []

for file in os.listdir(XML_DIR):
    if not file.endswith(".xml"):
        continue
    filepath = os.path.join(XML_DIR, file)
    print(f"Traitement {file} ...")

    with open(filepath, "rb") as f:  # lecture binaire pour gérer BOM
        content = f.read()

    # Supprimer BOM UTF-8 s’il existe
    if content.startswith(b'\xef\xbb\xbf'):
        content = content[3:]

    # Décoder en utf-8 et enlever espaces / lignes vides au début
    text = content.decode("utf-8").lstrip()

    # structures
    df_struct = extract_structures(text)
    all_structures.append(df_struct)

    # publications
    tree = ET.ElementTree(ET.fromstring(text))
    df_pub = extract_publis(tree, file)
    all_publis.append(df_pub)

# =====================================================================================================================================================
# CONCATÉNATION FINALE
# CREATION DE DEUX LISTES DISTINCTES
# - LISTE DES STRUCTURES D'AFFILIATIONS DES AUTEURS
# - LISTE DES AUTEURS
# =====================================================================================================================================================

# Structures uniques par id_Aurehal
df_structures = pd.concat(all_structures, ignore_index=True).drop_duplicates(subset="id_Aurehal")

# Publications cumulées
df_auteurs = pd.concat(all_publis, ignore_index=True)

# Supprimer les doublons exacts sur halID + author + id_Aurehal
df_auteurs = df_auteurs.drop_duplicates(subset=["halID", "author", "id_Aurehal"]).reset_index(drop=True)




print(f"\nNombre de structures uniques : {df_structures.shape[0]}")
print(f"Nombre de publications (lignes auteur) : {df_auteurs.shape[0]}")
print(f"Nombre de halID uniques : {df_auteurs['halID'].nunique()}")

# Temps de traitement pour toutes les années 2018-2025 : 30 sec. à 1m10

# df_auteurs.head(1)
# df_structures.head(1)

In [ ]:
#========================================================
# CROISEMENT STRUCTURES ET AUTEURS PAR L'ID AUREHAL
#=========================================================

df_auteurs_structures = ""
# # Merge en gardant toutes les lignes de df_publis
df_auteurs_structures = df_auteurs.merge(df_structures, on='id_Aurehal', how='left', suffixes=('', '_extra'))

# Suppression de la colonne en double
if 'id_Aurehal_extra' in df_auteurs_structures.columns:
    df_auteurs_structures = df_auteurs_structures.drop(columns=['id_Aurehal_extra'])

# Contrôles éventuels
# df_auteurs_structures.head(1)
# df_auteurs_structures[df_auteurs_structures["halID"] == "hal-01129393"]

In [ ]:
#==========================================================
# IDENTIFIER LES AUTEURS INRIA ET AJOUTER LEUR CENTRE
#==========================================================

#==========================================================
# Dictionnaire des centres et ID Aurehal
#==========================================================

# Dictionnaire id Aurehal → centres
codes_centres = {
    "419153": "Inria Univ. Rennes",
    "104751": "Inria Univ. Bordeaux",
    "34586": "Inria Univ. Cote Azur",
    "2497": "Inria Univ. Grenoble",
    "1096051": "Inria Lyon",
    "129671": "Inria Univ. Lorraine",
    "104752": "Inria Lille",
    "118511": "Inria Saclay",
    "454310": "Inria Paris",
    "1175218": "Inria Paris Sorbonne",
    "1225635": "Inria Saclay IPP",
    "1225627": "Inria Saclay UPS"
}

codes_set = set(codes_centres.keys())

#==========================================================
# Identifier les auteurs Inria
#==========================================================
def est_inria(value):
    if pd.isna(value) or value == "":
        return "non"
    # Séparer les codes (plusieurs codes peuvent être séparés par ",")
    codes_in_value = [v.strip() for v in value.split(",")]
    # Vérifier si un code de centre Inria est présent
    for code in codes_in_value:
            if code in codes_centres:
                return "oui"
    return "non"

# Appliquer sur id_Aurehal ou relations
df_auteurs_structures["Inria"] = df_auteurs_structures.apply(
    lambda row: est_inria(row["id_Aurehal"]) 
        if est_inria(row["id_Aurehal"]) == "oui" 
        else est_inria(row["relations"]),
    axis=1
)

# Pour contrôle : ici, FOCUS, situé en Italie, doit être taggé "Inria=oui"
# df_auteurs_structures[df_auteurs_structures["halID"] == "hal-02378761"] 

#==========================================================
# AJOUT DU CENTRE INRIA
#==========================================================

def get_centre(value):
    if pd.isna(value) or value == "":
        return None
    # Séparer les codes (dans relations, plusieurs codes séparés par ",")
    codes_in_value = [v.strip() for v in value.split(",")]
    # Chercher le premier code correspondant dans le dictionnaire
    for code in codes_in_value:
        if code in codes_centres:
            return codes_centres[code]
    return None

# Appliquer sur id_Aurehal ou relations
df_auteurs_structures["Centre"] = df_auteurs_structures.apply(
    lambda row: get_centre(row["id_Aurehal"]) or get_centre(row["relations"]),
    axis=1
)

# Temps de traitement : env. 4 à 24 sec
# Contrôles éventuels
# df_auteurs_structures[df_auteurs_structures["halID"] == "hal-01666389"]
# Flavio Oquendo doit être en Inria = non
# df_auteurs_structures[df_auteurs_structures["halID"] == "hal-01259762"]
# Boscain Ugo est à la fois Inria et autre chose, on va supprimer l'autre
# df_auteurs_structures[df_auteurs_structures["halID"] == "hal-01633660"]

# df_auteurs_structures[df_auteurs_structures["halID"] == "cel-01951107"]
# Khemakhem, Mohamed est Inria et aussi affiliation DE qui sera supprimée
# Laurent Romary est Inria et aussi affiliation DE qui sera supprimée

In [ ]:
#=========================================================
# Suppression des auteurs non Inria ayant double affiliation FR et INT
# Suppression des affiliations tierces des auteurs Inria
# Suppression des auteurs Inria n'appartenant pas à une équipe-projet
#==========================================================================

import pandas as pd

# Étape 1 : Supprimer les lignes Inria=non pour les auteurs ayant des affiliations en France et à l'étranger
df_inria_non = df_auteurs_structures[df_auteurs_structures["Inria"] == "non"]
grouped_non = df_inria_non.groupby(["halID", "author"])["country"].agg(set)
auteurs_mixtes_non = set(
    grouped_non[
        grouped_non.apply(lambda countries: any(c in fr_codes for c in countries) and any(c not in fr_codes for c in countries))
    ].index
)

mask_suppression_non_mixtes = (
    df_auteurs_structures.set_index(["halID", "author"]).index.isin(auteurs_mixtes_non) &
    (df_auteurs_structures["Inria"] == "non")
)

df_auteurs_structures_uniqaff = df_auteurs_structures[~mask_suppression_non_mixtes].reset_index(drop=True)

# Étape 2 : Supprimer les lignes Inria=non pour les auteurs ayant à la fois Inria=oui et Inria=non
grouped_inria = df_auteurs_structures_uniqaff.groupby(["halID", "author"])["Inria"].agg(set)
auteurs_avec_inria_oui_et_non = set(
    grouped_inria[
        grouped_inria.apply(lambda x: "oui" in x and "non" in x)
    ].index
)

mask_suppression_non_inria = (
    df_auteurs_structures_uniqaff.set_index(["halID", "author"]).index.isin(auteurs_avec_inria_oui_et_non) &
    (df_auteurs_structures_uniqaff["Inria"] == "non")
)

df_auteurs_structures_uniqaff = df_auteurs_structures_uniqaff[~mask_suppression_non_inria].reset_index(drop=True)

# Étape 3 : Pour les auteurs Inria=oui, ne garder que les lignes où type=researchteam
# Étape 3 : Supprimer toutes les lignes où Inria=oui et type != researchteam
mask_suppression_non_researchteam = (
    (df_auteurs_structures_uniqaff["Inria"] == "oui") &
    (df_auteurs_structures_uniqaff["type"] != "researchteam")
)

df_auteurs_inria_et_autres_pre = df_auteurs_structures_uniqaff[~mask_suppression_non_researchteam].reset_index(drop=True)

# Temps de traitement : env. 30 sec.

# Contrôle des modifications
# df_auteurs_inria_et_autres_pre[df_auteurs_inria_et_autres_pre["halID"] == "hal-01666389"]
# Flavio Oquendo doit être en Inria = non
# Khalil Drira (co-auteur FR) sera supprimé quand on séparera les auteurs Inria et les co-auteurs étrangers
# df_auteurs_inria_et_autres_pre[df_auteurs_inria_et_autres_pre["halID"] == "hal-01259762"]
# = Il devrait y avoir une seule ligne pour Boscain Ugo
# df_auteurs_inria_et_autres_pre[df_auteurs_inria_et_autres_pre["halID"] == "cel-01951107"]
# Une seule ligne pour Khemakhem Mohamed et Romary Laurent



In [ ]:
df_auteurs_inria_et_autres_pre.columns

In [ ]:
print(df_auteurs_inria_et_autres_pre["halID"].nunique())

In [ ]:
#==============================================================================================
# SUPPRIMER TOUTES LES PUBLICATIONS QUI N'ONT QUE DES AUTEURS INRIA
#==============================================================================================
df_auteurs_structures_inria_et_autres_int = df_auteurs_inria_et_autres_pre.copy()

print(f"Nbre de publications avant : {df_auteurs_structures_inria_et_autres_int["halID"].nunique()}")
df_auteurs_structures_inria_et_autres_final = ""


# 1. Identifier les halID "mixtes" (au moins un "oui" et un "non" pour Inria)
halID_mixtes = (
    df_auteurs_structures_inria_et_autres_int
    .groupby("halID")["Inria"]
    .apply(lambda x: ("oui" in x.values) and ("non" in x.values))
)
halID_mixtes = halID_mixtes[halID_mixtes].index

# 2. Filtrer le DataFrame pour ne garder que ces halID
df_auteurs_structures_inria_et_autres_final = df_auteurs_structures_inria_et_autres_int[
    df_auteurs_structures_inria_et_autres_int["halID"].isin(halID_mixtes)
].reset_index(drop=True)

print(f"Nbre de publications sans les publis uniquement Inria : {df_auteurs_structures_inria_et_autres_final["halID"].nunique()}")


# Aucun enregistrement ne devrait correspondre (publication uniquement Inria)
# df_auteurs_structures_inria_et_autres_final[df_auteurs_structures_inria_et_autres_final["halID"] == "tel-05202989"]

In [ ]:
#==============================================================================================
# CRER DEUX LISTES : AUTEURS INRIA ET AUTEURS INTERNATIONAUX
#==============================================================================================
import pandas as pd

# Liste des codes pays français à exclure
fr_codes = ['FR', 'GP', 'RE', 'MQ', 'GF', 'YT', 'PM', 'WF', 'TF', 'NC', 'PF']

# 1. Créer df_auteurs_inria : lignes où Inria = "oui"
df_auteurs_inria = df_auteurs_structures_inria_et_autres_final[
    df_auteurs_structures_inria_et_autres_final["Inria"] == "oui"
].reset_index(drop=True)

# 2. Créer df_auteurs_int : lignes où Inria = "non" ET country n'est pas dans fr_codes
df_auteurs_int = df_auteurs_structures_inria_et_autres_final[
    (df_auteurs_structures_inria_et_autres_final["Inria"] == "non") &
    (~df_auteurs_structures_inria_et_autres_final["country"].isin(fr_codes))
].reset_index(drop=True)

# Contrôle éventuel : une seule ligne auteur
# df_auteurs_int[df_auteurs_int["halID"] == "hal-01666389"]

# Nombre de copublications = nombre de Hal ID uniques dans les auteurs étrangers
# df_auteurs_int["halID"].nunique()


In [ ]:
#================================================================================================
# AJOUT DES DOMAINES ET MOTS-CLES INRIA AUX AUTEURS INRIA (accès réservé au fichier source : Bastri)
#=================================================================================================


  # Import du fichier des équipes-projet comportant les domaines et mots-clés
#================================================================================================

# 
df_bastri = pd.read_csv(
    "ExportBastri.csv",
    sep=';',           
    dtype=str,
    encoding='latin1', 
    on_bad_lines='skip'
)


# Nettoyage du fichier pour avoir une seule occurrence de chaque équipe-projet 

df_bastri_clean = (
    df_bastri
    .assign(has_domain=df_bastri["Domaine français"].notna())
    .sort_values("has_domain", ascending=False)
    .drop_duplicates(subset=["Num. national"], keep="first")
    .drop(columns="has_domain")
)

  # Enrichissement de la liste des auteurs Inria
#===========================================================================


cols_to_get = ["Domaine français", "Mots clés (français)"]

df_auteurs_inria_bastri = ""
# Merge : côté bastri Num. national, côté couples RNSR
df_auteurs_inria_bastri = df_auteurs_inria.merge(
    df_bastri_clean[["Num. national"] + cols_to_get],
    left_on="RNSR",        # colonne dans df_couples
    right_on="Num. national",
    how="left",
    suffixes=("", "_bastri")
)

# df_auteurs_inria_bastri.head(1)


In [ ]:
#==========================================================================================================================================================================
# POUR AUTEURS ETRANGERS :
# LISTE DES AFFILIATIONS DE NIVEAU SUPERIEUR (INSTITUTION OU REGROUPINSTITUTION)
#========================================================================================================================================================================
df_auteurs_int_et_institutions = ""
df_struct_institution_regroupinstit = ""

df_struct_institution_regroupinstit = df_structures[
    df_structures["type"].isin(["institution", "regroupinstitution"])
].copy()
df_struct_institution_regroupinstit.head(2)



# ====================================================================================================
# AJOUT D'UNE AFFILIATION DE NIVEAU SUPERIEUR AUX AUTEURS INTERNATIONAUX
# ====================================================================================================
df_rel = "" 
df_expl = "" 
df_priority = ""
# 1️⃣ Préparer le mapping id_Aurehal -> type et les infos à merger
dict_type = df_struct_institution_regroupinstit.set_index("id_Aurehal")["type"].to_dict()
df_infos = df_struct_institution_regroupinstit[["id_Aurehal", "name", "address","type", "statut", "ror_id"]].rename(
    columns={
        "id_Aurehal": "Id_Aurehal_org_Top_copubliant",
        "ror_id":"Ror_org_Top_copub",
        "name": "Nom_org_Top_copubliant",
        "address": "Adresse_org_Top_copubliant",
        "type":"Type_org_Top_copubliant",
        "statut": "Statut_org_Top_copubliant",
    }
)

# 2️⃣ Filtrer les lignes concernées (type != institution/regroupinstitution)
mask = ~df_auteurs_int["type"].isin(["institution", "regroupinstitution"])
df_rel = df_auteurs_int.loc[mask, ["relations"]].copy()
df_rel["orig_index"] = df_rel.index

# 3️⃣ Exploser les relations et ne garder que celles présentes dans df_struct_institution_regroupinstit
df_rel["relations_list"] = df_rel["relations"].fillna("").str.split(",")
df_expl = df_rel.explode("relations_list")
df_expl["relations_list"] = df_expl["relations_list"].str.strip()
df_expl = df_expl[df_expl["relations_list"].isin(dict_type)]
df_expl["type_relation"] = df_expl["relations_list"].map(dict_type)

# 4️⃣ Choisir l'id_Aurehal prioritaire (regroupinstitution > institution)
def choose_priority(g):
    if "regroupinstitution" in g["type_relation"].values:
        return g.loc[g["type_relation"]=="regroupinstitution", "relations_list"].iloc[0]
    elif "institution" in g["type_relation"].values:
        return g.loc[g["type_relation"]=="institution", "relations_list"].iloc[0]
    return None

# Priorité regroupinstitution > institution
df_expl["priority"] = df_expl["type_relation"].map({"regroupinstitution": 1, "institution": 2})
df_priority = df_expl.sort_values("priority").groupby("orig_index")["relations_list"].first().rename("Id_Aurehal_org_Top_copubliant")

#==================================
# ASSOCIER LES INSTITUTIONS DE NIVEAU SUPERIEUR AUX CO-AUTEURS INTERNATIONAUX
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# 5️⃣ Réassocier au df original
df_auteurs_int_et_institutions = df_auteurs_int.join(df_priority, how="left")

# 6️⃣ Ajouter nom et adresse de l'institution/ regroupinstitution
df_auteurs_int_et_institutions = df_auteurs_int_et_institutions.merge(
    df_infos,
    on="Id_Aurehal_org_Top_copubliant",
    how="left"
)


In [ ]:
#===========================================================================
# Regroupement auteurs Inria et auteurs étrangers par publication
#============================================================================
df_inria_r = ""
df_int_r = ""
df_copublis_final = ""
# renommage de certaines colonnes
df_inria_r = df_auteurs_inria_bastri.rename(columns={"author": "Auteur_inria"})
df_int_r = df_auteurs_int_et_institutions.rename(columns={"author": "Auteur_international"})

df_copublis_final = df_inria_r.merge(
    df_int_r,
    on="halID",
    how="inner",
    suffixes=("_halinria", "_int")
)


# Contrôles
# df_copublis_final[df_copublis_final["halID"] == "hal-01633660"][
#     ["Auteur_inria", "Auteur_international","ror_id_int"]]
# résultat attendu : Maugey, Thomas - Ma, Rui
 #                   Frossard, Pascal - Ma, Rui



# df_copublis_final[df_copublis_final["halID"] == "cel-01951107"][
#     ["Auteur_inria", "Auteur_international","ror_id_int"]]
# Résultat attendu : 
# Khemakhem, Mohamed - Gabay, Simon
# Romary, Laurent - Gabay, Simon

In [ ]:
#=======================================================
# Nettoyage après fusion
#=======================================================
# Nettoyage  (on supprime tous les espaces avant et après)
df_copublis_final = df_copublis_final.apply(
    lambda col: col.str.strip() if col.dtype == "object" else col
)

# Supprimer la colonne Num. national 
df_copublis_final = df_copublis_final.drop(columns=["Num. national"])

# Renommer les colonnes Domaine et mots-clés
df_copublis_final = df_copublis_final.rename(columns={
    "Domaine français": "Domaine_inria",
    "Mots clés (français)": "Mots_cles_inria"
})



In [ ]:
#=============================================================================
# Contrôle : Lignes où acronym est renseigné mais Domaine_inria est vide
# En principe sont vide : SED, affiliations directes à un Centre
#==============================================================================
df_sans_domaine = ""
df_sans_domaine = df_copublis_final[
    df_copublis_final["acronym_halinria"].notna() & df_copublis_final["Domaine_inria"].isna()
]


# Liste des valeurs pour lesquelles on n'a pas trouvé de domaine ou de mots-clés dans Bastri

df_grouped = (
    df_sans_domaine
    .groupby("acronym_halinria", as_index=False)
    .agg({
        "RNSR_halinria": lambda x: ", ".join(x.dropna().unique()),
        "Domaine_inria": lambda x: ", ".join(x.dropna().unique()),
        "halID": lambda x: ", ".join(x.dropna().unique())
    })
)

df_grouped
df_grouped.to_excel("equipes_sans_domaines.xlsx", index=False)



# Temps de traitement env 10 sec.

In [ ]:
#===========================================
# Ajout d'une colonne "UE/hors UE"
#===========================================

# Liste des codes pays UE ISO alpha-2
ue_codes = [
    "AT", "BE", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR",
    "DE", "GR", "HU", "IE", "IT", "LV", "LT", "LU", "MT", "NL",
    "PL", "PT", "RO", "SK", "SI", "ES", "SE"
]



# Ajouter la colonne "UE/Hors UE"
df_copublis_final["UE/Hors_UE"] = df_copublis_final["country_int"].apply(
    lambda x: "UE" if x in ue_codes else "Hors_UE"
)
print("UE/hors UE terminé")


In [ ]:
###########################################################
# Interprétation des codes Pays en noms en toutes lettres 
###########################################################

# Récupérer les données de l'API
# url = "https://restcountries.com/v3.1/all"
# response = requests.get(url)
# countries_data = response.json()
import requests
import pandas as pd

# --- Fonction pour récupérer le mapping code → nom ---
def get_country_mapping():
    url = "https://restcountries.com/v3.1/all"
    params = {"fields": "cca2,name"}
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()  # lève une erreur si le code HTTP n'est pas 200
        countries_data = response.json()
        
        if isinstance(countries_data, list):
            return {
                country.get("cca2"): country.get("name", {}).get("common")
                for country in countries_data
                if country.get("cca2") and "name" in country and "common" in country["name"]
            }
        else:
            print("⚠️ Format inattendu :", type(countries_data))
            return {}
    except Exception as e:
        print("❌ Erreur lors de la récupération des données pays :", e)
        return {}

# --- Récupération du mapping ---
country_mapping = get_country_mapping()
print("✅ Exemple : FR →", country_mapping.get("FR"))



# --- Ajout de la colonne avec le nom complet du pays ---
df_copublis_final["Nom_Pays_org_copubliant"] = df_copublis_final["country_int"].map(
    lambda code: country_mapping.get(code, code)  # garde le code si non trouvé
)

# --- Affichage pour vérifier ---
# print(df_couples_bastri)


In [ ]:
df_copublis_final.columns

In [ ]:

#=========================================================================================
# On renomme et on ordonne les colonnes, on supprime les colonnes inutiles ou redondantes
#=========================================================================================

# dictionnaire de renommage
rename_cols = {
    "halID" : "Hal_ID",
    "Type_doc":"DocType_halinria",
    "year_halinria": "Annee",
    "Centre_halinria": "Centre_inria",
    "acronym_halinria": "Equipe_inria",
    "Auteur_inria": "Auteur_Inria",
    "Auteur_international": "Auteur_etranger",
    "name_int": "Nom_org_copubliant",
    "address_int": "Adresse_org_copubliant",
    "type_int": "Type_org_copubliant",
    "Nom_org_Top_copubliant": "Nom_org_Top_copubliant",
    "Adresse_org_Top_copubliant": "Adresse_org_Top_copubliant",
    "Type_org_Top_copubliant": "Type_org_Top_copubliant",
    "country_int": "Code_Pays_orgs_copubliant",
    "Nom_Pays_org_copubliant" : "Nom_Pays_org_copubliant",
    "UE/Hors_UE": "UE/Hors_UE",
    "abstract_halinria": "Resume",
    "hal_domain_halinria": "Domaine_Hal",
    "keywords_halinria": "Mots_cles_Hal",
    "Domaine_inria": "Domaine_inria",
    "Mots_cles_inria": "Mots_cles_inria",
    "id_Aurehal_halinria": "Id_Aurehal_Equipe",
    "ror_id_int": "Ror_org_copubliant",
    "type_halinria": "Type_org_inria",
    "name_halinria": "Nom_equipe_inria",
    "address_halinria": "Adresse_equipe_inria",
    "RNSR_halinria": "RNSR_Equipe",
    "id_Aurehal_int": "id_Aurehal_org_copubliant",
    "statut_halinria": "statut_org_copubliant",
    "Ror_org_Top_copub" : "Ror_org_Top_copubliant",
    "Id_Aurehal_org_Top_copubliant": "id_Aurehal_org_Top_copubliant",
    "Statut_org_Top_copubliant": "Statut_org_Top_copubliant",
    "relations_int": "Autres_affiliations_copubliant",
    "xml_file_halinria": "Source"
}

# appliquer le renommage
df_copublis_final = df_copublis_final.rename(columns=rename_cols)

# réordonner les colonnes selon l'ordre voulu
df_copublis_final = df_copublis_final[list(rename_cols.values())]




# Suprression des colonnes inutiles
cols_to_drop = [
    "country_halinria",
    "relations_halinria",
    "year_int",
    "keywords_int",
    "hal_domain_int",
    "abstract_int",
    "xml_file_int",
    "acronym_int",
    "RNSR_int",
    "Centre_int"
]

df_copublis_final = df_copublis_final.drop(columns=[c for c in cols_to_drop if c in df_copublis_final.columns])

print("Renommage colonnes terminé")
# df_copublis_final.head(1)



In [ ]:
#========================================
# Suppression de toutes les lignes où le co-auteur n'a pas d'affiliation
#==================================================
print(len(df_copublis_final))
df_copublis_final = df_copublis_final[df_copublis_final["Nom_org_copubliant"].notna()]

print(len(df_copublis_final))


In [ ]:
#========================================
# Pour les org sans org TOP (tutelle), mais de type "institution", on recopie le nom et l'adresse
# dans les colonne des tutelles (Top)
#==================================================

import pandas as pd

df_copublis_final = df_copublis_final.copy()

# Nettoyer les colonnes ROR pour ne garder que l'identifiant
df_copublis_final['Ror_org_Top_copubliant'] = df_copublis_final['Ror_org_Top_copubliant'].str.replace(r'https?://ror\.org/', '', regex=True)
df_copublis_final['Ror_org_copubliant'] = df_copublis_final['Ror_org_copubliant'].str.replace(r'https?://ror\.org/', '', regex=True)


# 1. Remplir 'Nom_org_Top_copubliant' et 'Adresse_org_Top_copubliant' si 'Type_org_copubliant' est 'institution' ou 'regroupinstitution'
mask = (df_copublis_final['Type_org_copubliant'].isin(['institution', 'regroupinstitution'])) & (df_copublis_final['Nom_org_Top_copubliant'].isna())
df_copublis_final.loc[mask, 'Nom_org_Top_copubliant'] = df_copublis_final['Nom_org_copubliant']
df_copublis_final.loc[mask, 'Adresse_org_Top_copubliant'] = df_copublis_final['Adresse_org_copubliant']
df_copublis_final.loc[mask, 'Ror_org_Top_copubliant'] = df_copublis_final['Ror_org_copubliant']

# 2. Remplir l'adresse de org_Top avec celle de org_copubliant quand elle manque
mask2 = (df_copublis_final['Nom_org_copubliant'].notna()) & (df_copublis_final['Nom_org_Top_copubliant'].isna())
df_copublis_final.loc[mask2, 'Adresse_org_Top_copubliant'] = df_copublis_final['Adresse_org_copubliant']

# 3. Si le niveau Top n'a pas de ROR, on reprend celui de org
mask3 = (df_copublis_final['Ror_org_Top_copubliant'].isna())
df_copublis_final.loc[mask3, 'Ror_org_Top_copubliant'] = df_copublis_final['Ror_org_copubliant']


# 3. Créer un DataFrame pour les autres valeurs de 'Type_org_copubliant'
other_types = ((df_copublis_final['Nom_org_Top_copubliant'].isna()))
            
other_df = df_copublis_final[other_types].copy()


# Grouper par 'Nom_org_Top_copubliant' et concaténer les 'Hal_ID'
grouped = other_df.groupby('Nom_org_copubliant').agg({
    'Adresse_org_copubliant': 'first',
    'Type_org_copubliant': 'first',
    'id_Aurehal_org_copubliant':'first',
    'Hal_ID': lambda x: ', '.join(map(str, x)),
}).reset_index()

# Afficher les résultats

print("Recopie de org vers org TOP fait là où c'était nécessaire")

# Création du xlsx
df_copublis_final['Ror_org_Top_copubliant'] = df_copublis_final['Ror_org_Top_copubliant'].astype(str)
df_copublis_final['Ror_org_copubliant'] = df_copublis_final['Ror_org_copubliant'].astype(str)


# df_copublis_final.to_excel("copublications_Inria_2018-2024_sans_villes_a_nettoyer.xlsx", index=False)

# print("Fichier Excel 'copublications_Inria_2018-2024_sans_villes' généré")

print("Génération du fichier des institutions étrangères sans tutelles dans hal (Orgs_sans_Top)")
grouped.to_excel("orgs_sans_Top.xlsx", index=False)

# temps de traitement : 1 à 3m 

In [ ]:
#=============================================================================
# Suppression des variantes du titre autres qu'anglaises 
#===================================================================

import pandas as pd
import re

# Copie du DataFrame original
df_couples_adresses_eng = df_copublis_final.copy(deep=True)

def extraire_version_anglaise(nom):
    if pd.isna(nom):
        return nom

    # Vérifier si la valeur contient un "="
    if "=" not in nom:
        return nom

    # Mots-clés typiques des noms d'institutions en anglais
    mots_cles_anglais = [
        r'University', r'Institute', r'Center', r'of ', r'for ',
        r'Research', r'Lab', r'Department', r'School', r'College'
    ]

    # Convertir en chaîne de caractères
    nom = str(nom)

    # Chercher une sous-chaîne contenant un des mots-clés anglais
    for mot in mots_cles_anglais:
        pattern = re.compile(r'([^=]*{}[^=]*)'.format(mot), re.IGNORECASE)
        match = pattern.search(nom)
        if match:
            return match.group().strip()

    # Si aucun mot-clé trouvé, vérifier si une partie est en caractères latins
    parties = [p.strip() for p in nom.split('=')]
    for partie in parties:
        if re.search(r'^[A-Za-z0-9\s\[\]\,\.\-]+$', partie):
            return partie

    # Sinon, prendre la dernière partie après le dernier "="
    return parties[-1] if parties else nom

# Appliquer la fonction uniquement aux valeurs contenant un "="
mask = df_couples_adresses_eng['Nom_org_copubliant'].str.contains('=', na=False)
df_couples_adresses_eng.loc[mask, 'Nom_org_copubliant'] = df_couples_adresses_eng.loc[mask, 'Nom_org_copubliant'].apply(extraire_version_anglaise)

# Afficher le résultat
# print(df_couples_adresses_eng['Nom_org_copubliant'].head(10))




In [ ]:
# Supprimer toutes les valeurs Nan (pour le Dashboard)
df_couples_adresses_eng = df_couples_adresses_eng.fillna("")

# Enregistrement sous forme de fichier Excel
df_copublis_final_eng = df_couples_adresses_eng.copy()

df_copublis_final_eng.to_excel("copublications_Inria_2018-2024_sans_villes.xlsx", index=False)

# Temps de traitement : 2 à 4 mn

In [ ]:
#================================
# Suppression de tous les caractères invisibles
#======================================


def remove_invisible_chars(text):
    if not isinstance(text, str):
        return text
    return "".join(
        c for c in text
        if not unicodedata.category(c).startswith("C")
    )

# Appliquer à toutes les colonnes texte
df_couples_adresses = df_couples_adresses_eng.copy()
for col in df_couples_adresses.select_dtypes(include="object").columns:
    df_couples_adresses[col] = df_couples_adresses[col].apply(remove_invisible_chars)

# Temps de traitement : 40 secondes à 3 mn


In [ ]:
#================================
# Normalisation des apostrophes, guillemets et tirets typographiques
#======================================
TYPO_MAP = {
    # Apostrophes
    "\u2019": "'",
    "\u2018": "'",
    "\u02BC": "'",  # modifier letter apostrophe
    "\u02BB": "'",  # turned comma / okina
    # Guillemets
    "\u201C": '"',
    "\u201D": '"',
    # Tirets
    "\u2010": "-",  # hyphen
    "\u2011": "-",  # non-breaking hyphen
    "\u2012": "-",  # figure dash
    "\u2013": "-",  # en dash
    "\u2014": "-",  # em dash
    "\u2212": "-",  # minus sign
    # Virgules
    "\u060C": ",",  # Arabic comma → ASCII comma
    "\u2019": "'",  # apostrophe typographique
    "\u2018": "'",
    "\u2013": "-",  # en dash
    "\u2014": "-",  # em dash
}

def normalize_typography(text):
    if not isinstance(text, str):
        return text
    for k, v in TYPO_MAP.items():
        text = text.replace(k, v)
    return text

for col in df_couples_adresses.select_dtypes(include="object").columns:
    df_couples_adresses[col] = df_couples_adresses[col].apply(normalize_typography)

    # Temps de traitement 17 sec


In [ ]:
# Supprimer toutes les valeurs Nan (pour le Dashboard)
df_couples_adresses_eng = df_couples_adresses.fillna("")

# Enregistrement sous forme de fichier Excel
df_copublis_final_eng = df_couples_adresses_eng.copy()

df_copublis_final_eng.to_excel("copublications_Inria_2018-2024_sans_villes.xlsx", index=False)

# Temps de traitement : 2 à 4 mn